# Roll Cutting by Pattern Generation

### Imports

In [ ]:
# Add library location to system path
# This is only necessary when using a non-distributed version of amplpy_gurobi
from sys import path
path.append("D:\\Development\\ampl\solvers-private\\build\\vs64\\bin")


In [ ]:

# Import utilities
from math import floor, ceil


In [ ]:
# Import amplpy and gurobipy
from amplpy import AMPL
import amplpy_gurobi as ampls

### Register jupyter magics for AMPL

In [ ]:
from amplpy import register_magics
register_magics('_ampl_cells')  # Store %%ampl cells in the list _ampl_cells

### AMPL roll-cutting model (master problem)

In [ ]:
%%ampl
param nPatterns integer > 0;

set PATTERNS = 1..nPatterns;  # patterns
set WIDTHS;                   # finished widths

param order {WIDTHS} >= 0;    # rolls of width j ordered
param overrun;                # permitted overrun on any width

param rawWidth;               # width of raw rolls to be cut
param rolls {WIDTHS,PATTERNS} >= 0, default 0;   
                              # rolls of width i in pattern j

var Cut {PATTERNS} integer >= 0;  # raw rolls to cut in each pattern

minimize TotalRawRolls: sum {p in PATTERNS} Cut[p];

subject to OrderLimits {w in WIDTHS}:
  order[w] <= sum {p in PATTERNS} rolls[w,p] * Cut[p] <= order[w] + overrun;

### Python data for cutting patterns

In [ ]:
casename = 'UserCut3'

roll_width = 6000
overrun = 3

orders = {
1630: 172,
1625: 714,
1620: 110,
1617: 262,
1540: 32,
1529: 100,
1528: 76,
1505: 110,
1504: 20,
1484: 58,
1466: 15,
1450: 10,
1283: 40,
1017: 50,
970: 70,
930: 8,
916: 210,
898: 395,
894: 49,
881: 17,
855: 20,
844: 10,
805: 718,
787: 17,
786: 710,
780: 150,
754: 34,
746: 15,
707: 122,
698: 7,
651: 10,
644: 15,
638: 10,
605: 10,
477: 4,
473: 34,
471: 25,
468: 10,
460: 908,
458: 161,
453: 765,
447: 21,
441: 20,
422: 318,
421: 22,
419: 382,
396: 22,
309: 123,
266: 35
}

widths = list(sorted(orders.keys(), reverse=True))

### Define the AMPL master problem object and send data

In [ ]:
Master = AMPL()
Master.eval(_ampl_cells[0])

In [ ]:
# Send scalar values
Master.param['nPatterns'] = len(widths)
Master.param['overrun'] = overrun
Master.param['rawWidth'] = roll_width

# Send order vector
Master.set['WIDTHS'] = widths
Master.param['order'] = orders

# Generate and send initial pattern matrix
Master.param['rolls'] = {
    (widths[i], 1+i): int(floor(roll_width/widths[i]))
    for i in range(len(widths))
}

### AMPL pattern-generation model (subproblem)

In [ ]:
%%ampl
set SIZES;          # ordered widths
param cap >= 0;     # width of raw rolls
param val {SIZES};  # shadow prices of ordered widths (from Master)

var Qty {SIZES} integer >= 0;  # number of each width in generated pattern

maximize TotVal: sum {s in SIZES} val[s] * Qty[s];
    
subject to Cap: sum {s in SIZES} s * Qty[s] <= cap;

### Define the AMPL subproblem object and send data

In [ ]:
Sub = AMPL()
Sub.eval(_ampl_cells[1])

In [ ]:
Sub.set['SIZES'] = widths
Sub.param['cap'] = roll_width

### Set up for generation loop

In [ ]:
# Define which solver to use (right now either cplex or gurobi)
solver = "gurobi"

# Define a param for sending new patterns to master problem
Master.eval('param newPat {WIDTHS} integer >= 0;')

# Set solve options
Master.option['solver'] = solver
Master.option['relax_integrality'] =  1

Sub.option['solver'] = solver

### Loop

In [ ]:
while True:
    Master.solve()

    Sub.param['val'].setValues(Master.con['OrderLimits'].getValues())
    Sub.solve()
    if Sub.obj['TotVal'].value() <= 1.00001:
        break

    Master.param['newPat'].setValues(Sub.var['Qty'].getValues())
    Master.eval('let nPatterns := nPatterns + 1;')
    Master.eval('let {w in WIDTHS} rolls[w, nPatterns] := newPat[w];')

### Callback definition

In [ ]:
class MyCallback(ampls.GenericCallback):
    def __init__(self, stoprule):
        super(MyCallback, self).__init__()
        self._stoprule = stoprule
        self._current = 0
        self._continueOpt = True
        
    def setCurrentGap(self):
        print("Increasing gap tolerance to %.2f%%" % \
                    (100*self._stoprule['gaptol'][self._current]))
        ampls_model.setAMPLsParameter(ampls.SolverParams.DBL_MIPGap,
                             self._stoprule['gaptol'][self._current])
        self._current += 1
    def run(self):
        where = self.getAMPLWhere()
        # Check if we hit our runtime limit every time we enter a new MIP node
        if where == ampls.Where.MIPNODE:
            runtime = self.getValue(ampls.Value.RUNTIME).dbl
            if runtime >= self._stoprule['time'][self._current]:
                print(f"Stopping optimization at {runtime} seconds")
                self._continueOpt = True
                return -1
        # Print the solver message if present
        if where == ampls.Where.MSG:
            print(self.getMessage())
        return 0

### Invoke solver to find an integer solution

In [ ]:
Master.option['relax_integrality'] = 0
# Export model to ampls
# If we plan to import the results back to AMPL, we have to explicitly set what additional
# suffixes we want returned at this stage
# In this case, we want to return the mip gap as a suffix
ampls_model = Master.exportModel(solver, ["return_mipgap=5", "outlev=1"])

# Callback's stopping rule is created here...
stopdict = { 'time'   : (  3,    6,   10, 60 ),
             'gaptol' : ( .0002, .002, .02, .1 )
}
# ...and initialized in the constructor
callback = MyCallback(stopdict)
ampls_model.setCallback(callback)

# Invoke solver
# Most solvers (e.g. Gurobi https://support.gurobi.com/hc/en-us/articles/360039233191-How-do-I-change-parameters-in-a-callback-)
# do not support changing optimization parameters from a callback
# Instead we have to stop the optimization, change the desired parameters
# and start it again
while callback._continueOpt:
  callback._continueOpt = False
  ampls_model.optimize()
  if callback._continueOpt:
    callback.setCurrentGap()

# Import solution from Gurobi
Master.importSolution(ampls_model)

if solver == "gurobi": # use solver-specific API to get the actual MIPGAP
  print(f"From Gurobi MIPGap={100*ampls_model.getDoubleAttr(ampls.GRB_DBL_ATTR_MIPGAP):.3f}%")

print(f"From AMPL MIPGap={100*Master.getValue('TotalRawRolls.relmipgap'):.3f}%")
print(f"Objective value: {Master.getValue('TotalRawRolls')}")

### Retrieve solution

In [ ]:
# Prepare summary data
summary = {
    'Data': casename,
    'Obj': int(Master.obj['TotalRawRolls'].value()),
    'Waste': Master.getValue(
             'sum {p in PATTERNS} Cut[p] * \
                (rawWidth - sum {w in WIDTHS} w*rolls[w,p])'
             )
}

# Retrieve patterns and solution
npatterns = int(Master.param['nPatterns'].value())
rolls = Master.param['rolls'].getValues().toDict()
cutvec = Master.var['Cut'].getValues().toDict()

print("%d patterns generated" % (npatterns))

# Prepare solution data
solution = [
    ([int(rolls[widths[i], p+1]) 
      for i in range(len(widths))], int(cutvec[p+1]))
    for p in range(npatterns)
    if cutvec[p+1] > 0
][:50]

print("%d patterns used" % (len(solution)))

### Plotting routine

In [ ]:
def cuttingPlot(roll_width, widths, summary, solution):
    import numpy as np
    import matplotlib.pyplot as plt
    nchuncks = max(int(ceil(len(solution)/25.0)), 1)
    chunck_size = int(ceil(len(solution)/float(nchuncks)))
    start = 0
    for i in range(nchuncks):            
        plt.subplot(1, nchuncks, i+1)        
        sol = solution[start:start+chunck_size]
        start += chunck_size
        ind = np.arange(len(sol))
        acc = [0]*len(sol)
        colorlist = ['red','lightblue','orange','lightgreen',
                     'brown','fuchsia','silver','goldenrod']
        for p, (patt, rep) in enumerate(sol):
            for i in range(len(widths)):
                for j in range(patt[i]):
                    vec = [0]*len(sol)
                    vec[p] = widths[i]
                    plt.barh(ind, vec, 0.6, acc, 
                             color=colorlist[i%len(colorlist)], edgecolor='black')
                    acc[p] += widths[i]
        plt.xlim(0, roll_width)
        #plt.xticks(np.arange(0, roll_width, 10))
        plt.yticks(ind, tuple("x {:}".format(rep) for patt, rep in sol))
    plt.suptitle('{}: {} rolls, {} patterns, {:.2f}% waste'.format(
        summary['Data'],
        summary['Obj'],
        len(solution),
        round(100*summary['Waste']/(roll_width*summary['Obj']),2),
    ))   
    plt.show()

### Display solution

In [ ]:
# Create plot of solution
cuttingPlot(roll_width, widths, summary, solution)